<a href="https://colab.research.google.com/github/Arouoj/Investigating-Netflix-Movies/blob/main/ecommerce_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-groq langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [2]:
import os
from sqlalchemy import create_engine, Column, String, Float, Integer
from sqlalchemy.orm import declarative_base, sessionmaker
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain

from langchain_groq import ChatGroq
from google.colab import userdata, drive

from typing import List, Tuple, Any

In [3]:
# Mount Google Drive for persistent storage
drive.mount('/content/drive')

# Define database path in Google Drive
DB_PATH = "/content/drive/MyDrive/product_database/products.db"
os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
DATABASE_URL = f"sqlite:///{DB_PATH}"

Mounted at /content/drive


In [4]:
# Initialize database engine
engine = create_engine(DATABASE_URL, echo=True)

In [5]:
# Define Base class
Base = declarative_base()

# Define Product model
class Product(Base):
    __tablename__ = "products"

    id = Column(String, primary_key=True)
    name = Column(String, nullable=False)
    category = Column(String, nullable=False)
    price = Column(Float, nullable=False)
    description = Column(String)
    brand = Column(String, nullable=False)
    stock = Column(Integer, nullable=False)
    rating = Column(Float)

    def __repr__(self):
        return f"<Product(name='{self.name}', brand='{self.brand}', price={self.price})>"

In [6]:
def initialize_database():
    # Create table
    Base.metadata.create_all(engine)

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    # Check if products already exist
    if session.query(Product).first() is None:
        # Product data (20 products)
        products = [
            Product(id="P001", name="Wireless Bluetooth Headphones", category="Electronics", price=79.99,
                   description="High-quality wireless headphones with noise cancellation", brand="SoundMax", stock=45, rating=4.5),
            Product(id="P002", name="Smart Fitness Tracker", category="Wearables", price=49.99,
                   description="Track your steps, heart rate, and sleep patterns", brand="FitTech", stock=30, rating=4.3),
            Product(id="P003", name="Organic Cotton T-Shirt", category="Clothing", price=24.99,
                   description="100% organic cotton, sustainable fashion", brand="EcoWear", stock=100, rating=4.7),
            Product(id="P004", name="Professional Chef Knife", category="Kitchen", price=89.99,
                   description="High-carbon stainless steel chef knife", brand="ChefPro", stock=25, rating=4.8),
            Product(id="P005", name="Yoga Mat", category="Sports", price=29.99,
                   description="Non-slip, eco-friendly yoga mat", brand="YogaFlex", stock=60, rating=4.4),
            Product(id="P006", name="Coffee Maker", category="Kitchen", price=129.99,
                   description="Programmable coffee maker with thermal carafe", brand="BrewMaster", stock=20, rating=4.6),
            Product(id="P007", name="LED Desk Lamp", category="Home Office", price=39.99,
                   description="Adjustable LED lamp with wireless charging", brand="LightPro", stock=50, rating=4.2),
            Product(id="P008", name="Running Shoes", category="Sports", price=89.99,
                   description="Lightweight running shoes with cushioning", brand="SpeedRun", stock=40, rating=4.5),
            Product(id="P009", name="Wireless Mouse", category="Electronics", price=25.99,
                   description="Ergonomic wireless mouse with precision tracking", brand="TechGear", stock=80, rating=4.3),
            Product(id="P010", name="Noise-Canceling Earbuds", category="Electronics", price=99.99,
                   description="Premium earbuds with active noise canceling", brand="SoundElite", stock=35, rating=4.6),
            Product(id="P011", name="Smartwatch", category="Wearables", price=199.99,
                   description="Advanced smartwatch with fitness tracking and notifications", brand="TimeTech", stock=20, rating=4.7),
            Product(id="P012", name="Gaming Keyboard", category="Electronics", price=59.99,
                   description="Mechanical gaming keyboard with RGB lighting", brand="GamerPro", stock=50, rating=4.5),
            Product(id="P013", name="4K Monitor", category="Electronics", price=299.99,
                   description="27-inch 4K UHD monitor with high refresh rate", brand="VisionTech", stock=15, rating=4.8),
            Product(id="P014", name="Portable Power Bank", category="Accessories", price=39.99,
                   description="10000mAh fast-charging power bank", brand="ChargeMax", stock=75, rating=4.4),
            Product(id="P015", name="Wireless Charging Pad", category="Accessories", price=29.99,
                   description="Fast wireless charging pad for multiple devices", brand="QiCharge", stock=60, rating=4.3),
            Product(id="P016", name="Backpack for Laptops", category="Bags", price=49.99,
                   description="Water-resistant backpack with laptop compartment", brand="TravelGear", stock=90, rating=4.6),
            Product(id="P017", name="Mechanical Pencil Set", category="Office Supplies", price=19.99,
                   description="Premium mechanical pencils with extra refills", brand="WritePerfect", stock=100, rating=4.5),
            Product(id="P018", name="Wireless Security Camera", category="Home Security", price=129.99,
                   description="Full HD wireless security camera with night vision", brand="SafeHome", stock=30, rating=4.7),
            Product(id="P019", name="Electric Toothbrush", category="Personal Care", price=89.99,
                   description="Rechargeable electric toothbrush with multiple modes", brand="SmileCare", stock=55, rating=4.6),
            Product(id="P020", name="Air Purifier", category="Home Appliances", price=159.99,
                   description="HEPA air purifier for allergies and dust", brand="PureAir", stock=25, rating=4.8)
        ]

        # Add and commit data
        session.add_all(products)
        session.commit()
        print("Data inserted successfully!")
    else:
        print("Database already contains products.")

    session.close()

In [7]:
def clean_sql_query(query: str) -> str:
    """Clean the SQL query by removing formatting and fixing common issues"""
    # Remove 'SQLQuery:' prefix if present
    if 'SQLQuery:' in query:
        query = query.split('SQLQuery:', 1)[1]

    # Remove markdown SQL formatting
    query = query.replace('```sql', '').replace('```', '')

    # Remove quotes from identifiers
    query = query.replace('"id"', 'id')
    query = query.replace('"name"', 'name')
    query = query.replace('"category"', 'category')
    query = query.replace('"price"', 'price')
    query = query.replace('"description"', 'description')
    query = query.replace('"brand"', 'brand')
    query = query.replace('"stock"', 'stock')
    query = query.replace('"rating"', 'rating')

    # Strip whitespace and semicolon
    query = query.strip()
    if query.endswith(';'):
        query = query[:-1]

    return query

In [8]:
def setup_chatbot():
    # Initialize LLM
    GROQ_API_KEY = "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"
    if not GROQ_API_KEY:
        raise ValueError("GROQ_API_KEY not found in Colab userdata")

    llm = ChatGroq(
        model_name="llama3-70b-8192",
        groq_api_key=GROQ_API_KEY,
        temperature=0
    )

    # Connect LangChain to the database
    db = SQLDatabase.from_uri(DATABASE_URL)

    # Create SQL Chain using the new syntax
    chain = create_sql_query_chain(llm, db)
    return chain, db

In [9]:
def chat_with_sql(chain, db, query):
    try:
        # Generate SQL query
        sql_query = chain.invoke({"question": query})
        # Clean the SQL query
        cleaned_query = clean_sql_query(sql_query)
        # Execute the query
        result = db.run(cleaned_query)
        return f"Original Query: {query}\nSQL Query: {cleaned_query}\nResult: {result}"
    except Exception as e:
        return f"Error processing query: {str(e)}\nGenerated SQL: {sql_query}"

In [10]:
# Test Function
def main():
    # Initialize database and insert data
    initialize_database()

    # Setup chatbot
    try:
        chain, db = setup_chatbot()

        # Test queries
        test_queries = [
            "What is the cheapest product?",
            "Show me top rated electronics products.",
        ]

        for query in test_queries:
            print("\nQuery:", query)
            print("Bot Response:", chat_with_sql(chain, db, query))

    except Exception as e:
        print(f"Error setting up chatbot: {str(e)}")

# if __name__ == "__main__":
#     main()

In [11]:
def start_chatbot():
    print("Hello! I am your product assistant. You can start asking questions now.\n(Type 'exit' to end the chat.)")

    # Initialize the database and chatbot
    try:
        initialize_database()
        chain, db = setup_chatbot()

        while True:
            user_input = input("You: ")

            if user_input.lower() in ["exit", "quit", "q"]:
                print("Goodbye! Have a great day!")
                break

            result = chat_with_sql(chain, db, user_input)
            print("Bot:", result + "\n")

    except Exception as e:
        print(f"Error setting up chatbot: {str(e)}")


# Run the interactive chatbot
start_chatbot()

KeyboardInterrupt: Interrupted by user

Hello! I am your product assistant. You can start asking questions now.
(Type 'exit' to end the chat.)
2025-02-18 05:29:44,563 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-02-18 05:29:44,568 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("products")


...

INFO:sqlalchemy.engine.Engine:SELECT products.id AS products_id, products.name AS products_name, products.category AS products_category, products.price AS products_price, products.description AS products_description, products.brand AS products_brand, products.stock AS products_stock, products.rating AS products_rating 
FROM products
 LIMIT ? OFFSET ?


2025-02-18 05:29:44,588 INFO sqlalchemy.engine.Engine [cached since 2395s ago] (1, 0)


INFO:sqlalchemy.engine.Engine:[cached since 2395s ago] (1, 0)


Database already contains products.
2025-02-18 05:29:44,592 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [ ]:
# Question Samples (Be Specific):

# What's the cheapest product, and what is its description?
# How many products does the company sell, and how many in the stock?
# How many electronic products does the company sell, and what are their average rating?
# What products cost less than $50? just give the names, prices, and ratings.
# What categories does the company have? and what is the stock of each?
# What are the top rated brands? and what are their average prices?
# Show me all wireless products, with their description, and rating.